#NGS解析入門①A　(fastp:クオリティコントロール)


###東京医科歯科大学　医療データ科学推進室

###神沼英里

- 2019年7月20日初版
- 2020年6月16日改版

---
##目次




| | NGS解析入門①Aの実習 |プログラム言語　|
|:---|:---|:---|
|**[A]** |**Colabの設定**| |
|A-1|　Google Colabについて |Linux|
|A-2| Google Colabの準備作業| Linux|
|**[B]** |**NGS配列データをColabクラウドに取得**||  
|B-1|NGS配列データのダウンロード　 |Linux|
|B-2| ダウンロード済圧縮ファイルのサイズ確認 | Linux|
|B-3| ダウンロード済圧縮ファイルを解凍| Linux|
|B-4|解凍済ファイルの確認  | Linux|
|B-5| grep文字列検索で、NGSシーケンサ情報を確認   |Linux|
|B-6|  NGS配列のread長を調べる| Linux|
|**[C]** | **NGS配列のQC(クオリティコントロール)** ||
|C-1|　Quality Controlソフトウェア「fastp」のインストール |Linux|
|C-2|  fastpツールの実行権限確認| Linux|
|C-3|  fastpツールの実行| Linux|
|C-4|fastpツールの実行結果ファイルの確認|Linux|
|C-5|fastp実行ログ(htmlファイル)の表示|Python|
|C-6| QC filtering後の、FASTQファイルの行数確認 |Linux|



---
---
---
# [A] Colabの設定

###＜A-1: Google Colabについて＞


Colaboratory とは、機械学習の教育・研究を目的としたGoogle社クラウドで、無料でGPUを使えます（Linux実習では使用せず）。下記の特徴があります。






---
###＜A-2: Google Colabの準備作業＞

■準備作業

1.メニュの「ファイル」から「**ドライブにコピーを保存**」を選択。自分のGoogle Driveに**本JupyterNotebook(.ipynb)プログラムのコピーを保存する**。保存時に、ファイル名を変更しておく。他者のプログラムは閲覧のみ。自分のGoogleドライブ上のプログラムのみ、Colabクラウドで実行が可能。

2.ColabのJupyterNotebookは、「テキストセル」と「コードセル」の2種類がある。コードセルはプログラムを書き、実行できる。<font color="blue">実行時は▶ボタンを押す</font>。NGS解析入門のプログラムはほとんどがLinuxコマンドである。ColabのデフォルトはPython言語であるため、Linuxコマンドとシステムに認識させる為に1行実行の場合**「!」**マークをコマンド前に追加する。複数行のLinuxコマンドの場合は、先頭に**「%%bash」**を付ける。


■注意事項

- サーバへの接続制限（Colabは無料提供の為、下記条件で切断される。切断されたら、再接続ボタンをクリックする。）
              
■参考資料    

   - [Google Colaboratoryへようこそ](https://colab.research.google.com/notebooks/welcome.ipynb#)
   - [Qiita:【はやわかり＋保存版】Google Colabチートシート](https://qiita.com/tomo_makes/items/ec12dd61e4ced0a3dd66)
   
   

  ---

---
# [B] NGS配列データをColabクラウドに取得

SRA(Sequence Read Archive)は次世代シークエンサ配列のアーカイブデータベースです。SRAから配列データをダウンロードしてみましょう。


今回はデータサイズが約70Mbの溶血性レンサ球菌（ヒト病原菌）のWGS配列SRR496920を使います。ヒトゲノム配列のサイズ(3Gb=3000Mb)と比べると、ヒト配列が大きくて扱いにくい事が理解できるでしょう。

SRR496920データの特徴を下記にまとめます。

- Human pathogen
- [SRR496920](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR496920)
- Organism（生物種名） :  Streptococcus agalactiae MRI Z1-199
- Platform（DNAシークエンサ装置） : Illumina GA IIx
- Source : GENOMIC(ゲノム)
- Strategy : WGS(全ゲノム配列)
- Layout : Paired（ペア―ドエンド配列。read1とread2の2ファイルで構成される）

In [ ]:
#-------[B-1:NGS配列データのダウンロード]----------------------------------------------------------------------
#
# ＜SRAデータをColabクラウドに取得＞
# - demoディレクトリにcdで移動
# - SRAデータをftpサーバから取得します。
# - DRA(日本)からのデータダウンロードを推奨しますが、fastq形式は現在サポートが無いので、
#   欧州EBIからSRAをダウンロードします。
#
# ＜実行時ログ＞
# readデータ2つの取得に2分程かかります。次表示が出れば取得完了です。
# 途中経過ログは省略。
# 2018-10-24 06:58:16 (982 KB/s) - ‘SRR496920_2.fastq.gz’ saved [32605151]
# 2018-10-24 06:57:41 (5.00 MB/s) - ‘SRR496920_1.fastq.gz’ saved [30753763]
#
# ＜コマンドMEMO＞
# (22) wget=インターネット経由でデータ取得
#------------------------------------------------------------------
%%bash


wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR496/SRR496920/SRR496920_1.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR496/SRR496920/SRR496920_2.fastq.gz
ls

sample_data
SRR496920_1.fastq.gz
SRR496920_2.fastq.gz


--2020-08-03 14:08:05--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR496/SRR496920/SRR496920_1.fastq.gz
           => ‘SRR496920_1.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/SRR496/SRR496920 ... done.
==> SIZE SRR496920_1.fastq.gz ... 30753763
==> PASV ... done.    ==> RETR SRR496920_1.fastq.gz ... done.
Length: 30753763 (29M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  101K 4m56s
    50K .......... .......... .......... .......... ..........  0%  560K 2m55s
   100K .......... .......... .......... .......... ..........  0% 12.9M 1m57s
   150K .......... .......... .......... .......... ..........  0%  585K 1m40s
   200K .......... .......... .......... .......... ..........  0% 17.8M 80s
   250K .......... .....

In [ ]:
#-------[B-2:ダウンロード済圧縮ファイルのサイズ確認]-----------------------
#
#  - 取得ファイルが存在する事を、ファイル名確認コマンド「ls」で確認します。
#  - 「ls」はオプション「-l」を付けると、リスト表示になりファイルサイズや
#     タイムスタンプも確認できます。
#  - 両方とも30MB程のデータサイズと確認できます。
# [-rw-r--r-- 1 root root 30753763 Nov 26 07:00 demo/SRR496920_1.fastq.gz
# [-rw-r--r-- 1 root root 32605151 Nov 26 07:00 demo/SRR496920_2.fastq.gz]
#
# ＜コマンドMEMO＞
# (2) ls
#------------------------------------------------------------------
%%bash

ls -l ./*.gz

-rw-r--r-- 1 root root 30753763 Aug  3 14:08 ./SRR496920_1.fastq.gz
-rw-r--r-- 1 root root 32605151 Aug  3 14:08 ./SRR496920_2.fastq.gz


In [ ]:
#-------[B-3:ダウンロード済圧縮ファイルを解凍]-----------------------
#
#  - SRAデータは、fastqファイルがgzipコマンドで圧縮された状態
#  - gzipの解凍コマンドgunzipでファイルを解凍します。
#  - メタ文字*を使って、全ファイルを解凍しましょう。
#  - 解凍するとファイルサイズは、89Mになります（圧縮時に比べて3倍）。
#
# ＜コマンドMEMO＞
# (23) gunzip = gzip圧縮の解凍コマンド
#------------------------------------------------------------------

%%bash

gunzip *.gz

ls -l *.fastq

-rw-r--r-- 1 root root 89256698 Aug  3 14:08 SRR496920_1.fastq
-rw-r--r-- 1 root root 89256698 Aug  3 14:08 SRR496920_2.fastq


In [ ]:
#-------[B-4:解凍済ファイルの確認]-----------------------
#
#  - 何行か確認してみよう
#        ※read1, read2が同一行数である事が判ります。
#  - fastqデータの中身を確認してみよう
#     　※fastq形式は、4行で1配列を表します。
#
#         参考文献、https://ja.wikipedia.org/wiki/Fastq
#
#         1行目：文字「@」で始まるメタデータ。
#         2行目：塩基配列
#         3行目：文字「+」で始まるメタデータ
#         4行目：2行目配列のクオリティ値
#
#------------------------------------------------------------------

%%bash


wc -l ./*_1.fastq
wc -l ./*_2.fastq

echo '======================================================================='
head ./*_1.fastq

1395748 ./SRR496920_1.fastq
1395748 ./SRR496920_2.fastq
@SRR496920.1 SOLEXA1_0066_FC:4:1:2768:940/1
NCTGCGCTTTTCTGGATCCAGTATCATACTAACTGATTGCTCAATTGCTTCTAATTCTTGTTCAGCTGACACACGGTTGAGTGCTAAGCGTTCAACTGGC
+
#++++53555@@22@@@@@@88888:<<<<1777758588@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@7@@@@@@@@@:::1:13333@@8@@<::7:
@SRR496920.2 SOLEXA1_0066_FC:4:1:10013:937/1
NAATGATAAAATAGTTGAAAATGATGTTTTAAGCCATCAGATAAATAGATACGGTCATCTTTAGTGATGATAATGCCTTCAATGTAGGAAACAGCGTTTA
+
#1121777752222CC@C@@@CC@@CC@@22@@@@@C@@@@@@@@<<<:<@@@C@:<<<<@C@@@C@C@@@@@C@@@@@@@@@@@@C@@@@8@@8<<<<:
@SRR496920.3 SOLEXA1_0066_FC:4:1:12315:949/1
NGCAAATGCCATCTACGTACAACAATACTTCCTCAATCCTTTGGCTAAACTCTTCCTGTGAAAGTCCTTTATCGATATAGTCTAAAGCTGATACCTGATA


In [ ]:
#-------[B-5: grep文字列検索で、NGSシーケンサ情報を確認 ]-----------------------
#
#  grepコマンドは、文字列検索で使われます。fastq配列データは4行1配列でした。
#  4行のうち、1行目の@の後の文字列は、配列IDやDNAシークエンサ情報です。
#  まず、SRR番号を含む行を抽出してみましょう。
#  Colabの初期設定で、データが長すぎるとエラーになるので、headで最初の120行を抽出します。
#  「> 保存ファイル名」は、標準出力をファイルに保存する方法でしたね（3-1参照）。
#   続いて保存ファイルのSRRを含む行を、grepで検索します。
#
#  　※4行1配列なので、丁度30行のデータが検索できました。
# 　 ※1配列ごとに、SRR496920.1と番号が付いている事が判ります。
#
#
# ＜コマンドMEMO＞
# (24) grep = 文字列検索, -nオプションは行番号表示
#
#------------------------------------------------------------------

%%bash

head -120 ./*_1.fastq > ./*_1_120.fastq
grep -n "SRR"  ./*_1_120.fastq

1:@SRR496920.1 SOLEXA1_0066_FC:4:1:2768:940/1
5:@SRR496920.2 SOLEXA1_0066_FC:4:1:10013:937/1
9:@SRR496920.3 SOLEXA1_0066_FC:4:1:12315:949/1
13:@SRR496920.4 SOLEXA1_0066_FC:4:1:19257:930/1
17:@SRR496920.5 SOLEXA1_0066_FC:4:1:8730:960/1
21:@SRR496920.6 SOLEXA1_0066_FC:4:1:9316:964/1
25:@SRR496920.7 SOLEXA1_0066_FC:4:1:10463:961/1
29:@SRR496920.8 SOLEXA1_0066_FC:4:1:12391:957/1
33:@SRR496920.9 SOLEXA1_0066_FC:4:1:7392:975/1
37:@SRR496920.10 SOLEXA1_0066_FC:4:1:17589:974/1
41:@SRR496920.11 SOLEXA1_0066_FC:4:1:2536:978/1
45:@SRR496920.12 SOLEXA1_0066_FC:4:1:2366:999/1
49:@SRR496920.13 SOLEXA1_0066_FC:4:1:8937:996/1
53:@SRR496920.14 SOLEXA1_0066_FC:4:1:9159:988/1
57:@SRR496920.15 SOLEXA1_0066_FC:4:1:12973:993/1
61:@SRR496920.16 SOLEXA1_0066_FC:4:1:13576:1000/1
65:@SRR496920.17 SOLEXA1_0066_FC:4:1:15984:990/1
69:@SRR496920.18 SOLEXA1_0066_FC:4:1:3380:1004/1
73:@SRR496920.19 SOLEXA1_0066_FC:4:1:11056:1007/1
77:@SRR496920.20 SOLEXA1_0066_FC:4:1:13387:1004/1
81:@SRR496920.21 SOLEXA1_0066_FC:4:1:

In [ ]:
#-------[B-6: NGS配列のread長を調べる]-----------------------------------
#
# 　SRR496920データは、イルミナ社のSOLEXAシークエンサの解読配列でした。
#   イルミナ社のシークエンサは、配列長は固定です。Read1ファイル(SRR496920_1.fastq)で
#   最初に出てくる配列を文字列として取得し、文字列の長さを調べる事で、
#   SRR496920データの配列長を確認してみましょう。
#
# ＜コマンドMEMO＞
# (F) メタ文字（``) = バッククオーテーションで囲んだコマンドを実行する。
# (25) expr = 四則演算などの式を評価
# (14, 15) head/tail = ファイルの先頭/末尾を取得
# (E) メタ文字（|) = パイプ
#
#------------------------------------------------------------------

%%bash

# 2行目のみを取得して、tmpstr.txtに保存する。

head -2 ./SRR496920_1.fastq | tail -1  > tmpstr.txt
SEQ=`cat tmpstr.txt`
echo "[1] SEQUENCE = " $SEQ
echo "[2] READ LENGTH = "  `expr length $SEQ`


[1] SEQUENCE =  NCTGCGCTTTTCTGGATCCAGTATCATACTAACTGATTGCTCAATTGCTTCTAATTCTTGTTCAGCTGACACACGGTTGAGTGCTAAGCGTTCAACTGGC
[2] READ LENGTH =  100


---
---
# [C] NGS配列のQC(クオリティコントロール)


- FASTQ形式データの4行目には、NGS配列の品質を表す品質情報（クオリティ）が記載されています。NGS配列のクオリティ情報は、DNAシークエンサの塩基の解読品質を表します。この配列品質の評価ツールとして、fastpツールが最近使われています。ここでは、[fastpツール](https://academic.oup.com/bioinformatics/article/34/17/i884/5093234)をインストールして、前述のfastqファイルの品質結果を表示してみましょう。

- fastpは、低品質のリードを削除（フィルタリング）します。これをQC filteringと言い、変異同定解析・機能注釈などの本格的な解析前に、「前処理」として実行します。

In [ ]:
#-------[C-1: Quality Controlソフトウェア「fastp」のインストール]-----------------------
#
# fastpツールのインストール方法
#  ①binaryファイルダウンロード
#  ②実行権限付与
#
#------------------------------------------------

%%bash

wget http://opengene.org/fastp/fastp
chmod +x ./fastp

--2020-08-03 14:23:51--  http://opengene.org/fastp/fastp
Resolving opengene.org (opengene.org)... 8.210.133.117
Connecting to opengene.org (opengene.org)|8.210.133.117|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10444776 (10.0M) [application/octet-stream]
Saving to: ‘fastp’

     0K .......... .......... .......... .......... ..........  0%  126K 80s
    50K .......... .......... .......... .......... ..........  0%  252K 60s
   100K .......... .......... .......... .......... ..........  1%  231M 40s
   150K .......... .......... .......... .......... ..........  1%  285M 30s
   200K .......... .......... .......... .......... ..........  2%  253K 32s
   250K .......... .......... .......... .......... ..........  2%  230M 26s
   300K .......... .......... .......... .......... ..........  3%  278M 22s
   350K .......... .......... .......... .......... ..........  3%  253K 24s
   400K .......... .......... .......... .......... ..........  4%  295M 21s
  

In [ ]:
#-------[C-2: fastpツールの実行権限確認]-----------------------
#
# fastpツールの実行権限確認と、バージョン表示
#  (-v オプションでバージョンが表示される)
#------------------------------------------------

%%bash

ls -l fastp

./fastp -v

-rwxr-xr-x 1 root root 10444776 Jun 19 07:30 fastp


fastp 0.21.0


In [ ]:
#-------[C-3: fastpツールの実行]-----------------------
#
# fastpツールの実行
#  入力はpaired-endファイルなので、出力もout_1.fqとout_2.fqとする。
#  .fqは.fastqの省略形
#---------------------------------------------------------------

%%bash

./fastp -i ./*_1.fastq -I ./*_2.fastq -o out_1.fq -O out_2.fq

Read1 before filtering:
total reads: 348937
total bases: 34893700
Q20 bases: 32524122(93.2092%)
Q30 bases: 30388732(87.0895%)

Read2 before filtering:
total reads: 348937
total bases: 34893700
Q20 bases: 32681488(93.6601%)
Q30 bases: 30066178(86.1651%)

Read1 after filtering:
total reads: 323501
total bases: 32341182
Q20 bases: 31096635(96.1518%)
Q30 bases: 29254669(90.4564%)

Read2 aftering filtering:
total reads: 323501
total bases: 32341182
Q20 bases: 31135893(96.2732%)
Q30 bases: 28917976(89.4153%)

Filtering result:
reads passed filter: 647002
reads failed due to low quality: 46116
reads failed due to too many N: 4756
reads failed due to too short: 0
reads with adapter trimmed: 468
bases trimmed due to adapters: 18204

Duplication rate: 0.822091%

Insert size peak (evaluated by paired-end reads): 169

JSON report: fastp.json
HTML report: fastp.html

./fastp -i ./SRR496920_1.fastq -I ./SRR496920_2.fastq -o out_1.fq -O out_2.fq 
fastp v0.21.0, time used: 6 seconds


In [ ]:
#-------[C-4: fastpツールの実行結果ファイルの確認]-----------------------
#
# fastp実行結果の生成ファイル名の表示
#
#    fastp.htmlが結果報告書
#
#---------------------------------------------------------------
%%bash

ls -l

total 346712
-rw-r--r-- 1 root root     7491 Aug  3 14:10 *_1_120.fastq
-rwxr-xr-x 1 root root 10444776 Jun 19 07:30 fastp
-rw-r--r-- 1 root root   451818 Aug  3 14:25 fastp.html
-rw-r--r-- 1 root root   109503 Aug  3 14:25 fastp.json
-rw-r--r-- 1 root root 82742592 Aug  3 14:25 out_1.fq
-rw-r--r-- 1 root root 82742592 Aug  3 14:25 out_2.fq
drwxr-xr-x 1 root root     4096 Jul 30 16:30 sample_data
-rw-r--r-- 1 root root 89256698 Aug  3 14:08 SRR496920_1.fastq
-rw-r--r-- 1 root root 89256698 Aug  3 14:08 SRR496920_2.fastq
-rw-r--r-- 1 root root      101 Aug  3 14:10 tmpstr.txt


In [ ]:
#-------[C-5: fastp実行ログ(htmlファイル)の表示]-----------------------
#
# fastp.htmlのfastp分析結果を表示
# ここは、pythonコマンドを使います。
#
#---------------------------------------------------------------

import codecs
import IPython.display as d


f=codecs.open("fastp.html", 'r')
d.display_html(f.read(), raw=True)


fastp version:,0.21.0 (https://github.com/OpenGene/fastp)
sequencing:,paired end (100 cycles + 100 cycles)
mean length before filtering:,"100bp, 100bp"
mean length after filtering:,"99bp, 99bp"
duplication rate:,0.822091%
Insert size peak:,169
total reads:,697.874000 K
total bases:,69.787400 M
Q20 bases:,65.205610 M (93.434646%)
Q30 bases:,60.454910 M (86.627256%)
GC content:,35.695674%


In [ ]:
#----[C-6: QC filtering後の、FASTQファイルの行数確認]-----------------
#
# （※フィルタリング後のデータ量は92.7%である事に注意）
#
#---------------------------------------------------------------------------
%%bash

wc -l ./SRR*_1.fastq
wc -l ./out_1.fq
echo "-----------------"
wc -l ./SRR*_2.fastq
wc -l ./out_2.fq


1395748 ./SRR496920_1.fastq
1294004 ./out_1.fq
-----------------
1395748 ./SRR496920_2.fastq
1294004 ./out_2.fq


---
---
[NGS解析入門①B.ipynb](https://colab.research.google.com/drive/1QduoGymwF4Y0BZE3TfDUVfXhNo2qiTFy)へ続く
